# Étape 0 : Installation des dépendances

In [1]:
!pip install -U \
  ultralytics>=8.1.0 \
  roboflow \
  transformers \
  datasets \
  torchvision \
  paddlepaddle \
  paddleocr \
  spacy \
  fuzzywuzzy \
  opencv-python-headless

!python -m spacy download fr_core_news_md
!pip uninstall -y opencv-python

!pip install -U opencv-python-headless


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 14.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (50.0 MB)
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.10.0.84
    Uninstalling opencv-python-headless-4.10.0.84:
      Successfully uninstalled opencv-pyt

In [2]:
import os
import shutil
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from ultralytics.nn.tasks import DetectionModel
from paddleocr import PaddleOCR
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from fuzzywuzzy import process
import spacy
from datasets import load_dataset
from roboflow import Roboflow
import matplotlib.pyplot as plt

/usr/local/lib/python3.11/dist-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


# Étape 2 : Téléchargement des données

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
# Définir le chemin de base
DATASET_PATH = "/content/drive/MyDrive/pi_project_3"
os.makedirs(DATASET_PATH, exist_ok=True)


In [33]:
# Téléchargement des données via Roboflow


from roboflow import Roboflow

rf = Roboflow(api_key="m3zcgsb8zN6fHOST8Swy")
project = rf.workspace("doxariaproject").project("eya_-2bh")
version = project.version(6)
dataset = version.download("yolov8")


# Copier les données dans le dossier de travail
dataset_dir = os.path.join(DATASET_PATH, "datasets")
os.makedirs(dataset_dir, exist_ok=True)
shutil.copytree(dataset.location, dataset_dir, dirs_exist_ok=True)



loading Roboflow workspace...
loading Roboflow project...


'/content/drive/MyDrive/pi_project_3/datasets'

# Création du fichier data.yaml

In [6]:

YOLO_DATA_YAML = os.path.join(DATASET_PATH, "data.yaml")
yaml_content = f"""train: {os.path.join(dataset_dir, 'train/images')}
val: {os.path.join(dataset_dir, 'valid/images')}
test: {os.path.join(dataset_dir, 'test/images')}

nc: 2
names:
  0: dosage
  1: medicament
"""
with open(YOLO_DATA_YAML, 'w') as f:
    f.write(yaml_content)

print("Contenu de data.yaml :")
!cat {YOLO_DATA_YAML}

Contenu de data.yaml :
train: /content/drive/MyDrive/pi_project_3/datasets/train/images
val: /content/drive/MyDrive/pi_project_3/datasets/valid/images
test: /content/drive/MyDrive/pi_project_3/datasets/test/images

nc: 2
names:
  0: dosage
  1: medicament


## Verification des dossiers

In [7]:
import os

DATASET_PATH = "/content/drive/MyDrive/pi_project_3/datasets"
for split in ['train', 'valid', 'test']:
    img_dir = os.path.join(DATASET_PATH, split, 'images')
    label_dir = os.path.join(DATASET_PATH, split, 'labels')
    print(f"\nVérification du dossier {split}:")
    print(f"Images : {len([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])}")
    print(f"Labels : {len([f for f in os.listdir(label_dir) if f.endswith('.txt')])}")


Vérification du dossier train:
Images : 2916
Labels : 2916

Vérification du dossier valid:
Images : 240
Labels : 240

Vérification du dossier test:
Images : 233
Labels : 233


# Étape 4 : Entraînement de YOLOv8l

In [10]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu12

Looking in indexes: https://download.pytorch.org/whl/cu12


In [11]:
import torch
torch.cuda.empty_cache()


In [ ]:
from ultralytics import YOLO

# Load the pre-trained YOLOv8 model
model = YOLO('yolov8l.pt')  # Large model, suitable for handwritten quality

# Train the model
model.train(
    data='/content/drive/MyDrive/pi_project_3/data.yaml',
    epochs=60,   # 60 epochs as desired
    batch=16,
    lr0=0.001,
    lrf=0.1,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=5,
    optimizer='AdamW',
    degrees=5,
    translate=0.1,
    scale=0.7,
    shear=2.0,
    hsv_h=0.005,
    hsv_s=0.5,
    hsv_v=0.2,
    flipud=0.0,
    fliplr=0.1,
    name='yolo_medicament_dosage_60epochs'
)

# Evaluate the model on the test set
metrics = model.val(data='/content/drive/MyDrive/pi_project_3/data.yaml', split='test')
print(f"mAP@50: {metrics.box.map50:.3f}, mAP@50:95: {metrics.box.map:.3f}")


100%|██████████| 83.7M/83.7M [00:00<00:00, 115MB/s]


Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/MyDrive/pi_project_3/data.yaml, epochs=60, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo_medicament_dosage_60epochs, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

100%|██████████| 755k/755k [00:00<00:00, 20.4MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 84.6MB/s]


...

                   all        240        819      0.695      0.602      0.621      0.306
                dosage        101        236      0.623      0.436      0.454      0.203
            medicament        238        583      0.767      0.768      0.788      0.408
Speed: 0.3ms preprocess, 16.2ms inference, 0.0ms loss, 4.5ms postprocess per image
Results saved to runs/detect/yolo_medicament_dosage_60epochs
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 112 layers, 43,608,150 parameters, 0 gradients, 164.8 GFLOPs


val: Scanning /content/drive/MyDrive/pi_project_3/datasets/test/labels... 233 images, 0 backgrounds, 0 corrupt: 100%|██████████| 233/233 [00:02<00:00, 107.22it/s]


val: New cache created: /content/drive/MyDrive/pi_project_3/datasets/test/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:11<00:00,  1.33it/s]


                   all        233        750      0.831      0.601      0.694       0.34
                dosage        100        217      0.784      0.447      0.551      0.257
            medicament        233        533      0.877      0.754      0.836      0.423
Speed: 2.1ms preprocess, 36.9ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/yolo_medicament_dosage_60epochs2
mAP@50: 0.694, mAP@50:95: 0.340
